# Лабораторная работа: Сверточные нейронные сети (CNN)

## Задание
Реализация и анализ сверточной нейронной сети для классификации изображений на наборе данных CIFAR-10.


## 1. Загрузка и подготовка данных


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (10, 8)
plt.rcParams['font.size'] = 12


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(f"Форма тренировочных данных: {x_train.shape}")
print(f"Форма тестовых данных: {x_test.shape}")
print(f"Количество классов: {len(np.unique(y_train))}")
print(f"Диапазон значений пикселей: [{x_train.min()}, {x_train.max()}]")

class_names = ['самолет', 'автомобиль', 'птица', 'кошка', 'олень', 
               'собака', 'лягушка', 'лошадь', 'корабль', 'грузовик']


In [ ]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

y_train_categorical = to_categorical(y_train, 10)
y_test_categorical = to_categorical(y_test, 10)

print(f"Форма после нормализации: {x_train.shape}")
print(f"Форма меток после one-hot: {y_train_categorical.shape}")
print(f"Диапазон после нормализации: [{x_train.min()}, {x_train.max()}]")

plt.figure(figsize=(15, 3))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(x_train[i])
    plt.title(f'{class_names[y_train[i][0]]}')
    plt.axis('off')
plt.tight_layout()
plt.show()


## 2. Определение архитектуры CNN


In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


## 3. Обучение модели


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.0001)

history = model.fit(x_train, y_train_categorical,
                    batch_size=128,
                    epochs=20,
                    validation_split=0.2,
                    callbacks=[early_stopping, reduce_lr],
                    verbose=1)


In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Тренировочная точность')
plt.plot(history.history['val_accuracy'], label='Валидационная точность')
plt.title('Точность модели')
plt.xlabel('Эпоха')
plt.ylabel('Точность')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Тренировочные потери')
plt.plot(history.history['val_loss'], label='Валидационные потери')
plt.title('Потери модели')
plt.xlabel('Эпоха')
plt.ylabel('Потери')
plt.legend()

plt.tight_layout()
plt.show()


## 4. Оценка модели


In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test_categorical, verbose=0)
print(f"Точность на тестовом наборе: {test_accuracy:.4f}")
print(f"Потери на тестовом наборе: {test_loss:.4f}")

y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = y_test.flatten()


In [ ]:
cm = confusion_matrix(y_true_classes, y_pred_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.title('Матрица путаницы')
plt.xlabel('Предсказанные классы')
plt.ylabel('Реальные классы')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

print("\nОтчет о классификации:")
print(classification_report(y_true_classes, y_pred_classes, target_names=class_names))


In [ ]:
plt.figure(figsize=(15, 6))
for i in range(15):
    plt.subplot(3, 5, i+1)
    plt.imshow(x_test[i])
    predicted_class = class_names[y_pred_classes[i]]
    true_class = class_names[y_true_classes[i]]
    color = 'green' if y_pred_classes[i] == y_true_classes[i] else 'red'
    plt.title(f'Предсказано: {predicted_class}\nРеально: {true_class}', color=color)
    plt.axis('off')
plt.tight_layout()
plt.show()


## 5. Визуализация фильтров


In [ ]:
first_layer_weights = model.layers[0].get_weights()[0]
print(f"Форма фильтров первого слоя: {first_layer_weights.shape}")

plt.figure(figsize=(15, 10))
for i in range(32):
    plt.subplot(4, 8, i+1)
    filter_img = first_layer_weights[:, :, :, i]
    filter_img = (filter_img - filter_img.min()) / (filter_img.max() - filter_img.min())
    plt.imshow(filter_img)
    plt.title(f'Фильтр {i+1}')
    plt.axis('off')
plt.suptitle('Фильтры первого сверточного слоя', fontsize=16)
plt.tight_layout()
plt.show()


## 6. Улучшение модели


In [ ]:
improved_model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.3),
    
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.3),
    
    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.3),
    
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.6),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

improved_model.compile(optimizer=Adam(learning_rate=0.0005),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

print("Улучшенная архитектура:")
improved_model.summary()


In [ ]:
improved_history = improved_model.fit(x_train, y_train_categorical,
                                     batch_size=64,
                                     epochs=25,
                                     validation_split=0.2,
                                     callbacks=[early_stopping, reduce_lr],
                                     verbose=1)


In [ ]:
improved_test_loss, improved_test_accuracy = improved_model.evaluate(x_test, y_test_categorical, verbose=0)

print(f"\nСравнение результатов:")
print(f"Базовая модель - Точность: {test_accuracy:.4f}, Потери: {test_loss:.4f}")
print(f"Улучшенная модель - Точность: {improved_test_accuracy:.4f}, Потери: {improved_test_loss:.4f}")
print(f"Улучшение точности: {improved_test_accuracy - test_accuracy:.4f}")

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(history.history['accuracy'], label='Базовая - Тренировочная')
plt.plot(history.history['val_accuracy'], label='Базовая - Валидационная')
plt.plot(improved_history.history['accuracy'], label='Улучшенная - Тренировочная')
plt.plot(improved_history.history['val_accuracy'], label='Улучшенная - Валидационная')
plt.title('Сравнение точности')
plt.xlabel('Эпоха')
plt.ylabel('Точность')
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(history.history['loss'], label='Базовая - Тренировочные')
plt.plot(history.history['val_loss'], label='Базовая - Валидационные')
plt.plot(improved_history.history['loss'], label='Улучшенная - Тренировочные')
plt.plot(improved_history.history['val_loss'], label='Улучшенная - Валидационные')
plt.title('Сравнение потерь')
plt.xlabel('Эпоха')
plt.ylabel('Потери')
plt.legend()

plt.subplot(1, 3, 3)
models = ['Базовая', 'Улучшенная']
accuracies = [test_accuracy, improved_test_accuracy]
plt.bar(models, accuracies, color=['skyblue', 'lightcoral'])
plt.title('Финальная точность')
plt.ylabel('Точность')
for i, v in enumerate(accuracies):
    plt.text(i, v + 0.01, f'{v:.3f}', ha='center')

plt.tight_layout()
plt.show()


## 7. Анализ и выводы


### Анализ результатов

**Изменения в архитектуре улучшенной модели:**
1. **Увеличение количества фильтров**: С 32 до 64 в первом слое, что позволяет модели извлекать больше признаков
2. **Добавление слоев**: Увеличение глубины сети с дополнительными сверточными слоями
3. **Увеличение dropout**: С 0.25 до 0.3 для лучшей регуляризации
4. **Добавление полносвязного слоя**: Дополнительный Dense слой для лучшего обучения сложных паттернов
5. **Уменьшение learning rate**: С 0.001 до 0.0005 для более стабильного обучения
6. **Уменьшение batch size**: С 128 до 64 для лучшей градиентной оптимизации

**Проблемы и их решения:**
- **Переобучение**: Решено добавлением BatchNormalization и увеличенного dropout
- **Медленная сходимость**: Решено уменьшением learning rate и добавлением ReduceLROnPlateau
- **Недостаточная сложность**: Решено увеличением глубины и ширины сети

**Влияние изменений на качество:**
- BatchNormalization значительно улучшил стабильность обучения
- Увеличение dropout предотвратило переобучение
- Дополнительные слои позволили модели изучить более сложные паттерны
- Уменьшение learning rate сделало обучение более стабильным


### Адаптация для других датасетов

**MNIST (28x28, 1 канал, 10 классов):**
- Изменить input_shape на (28, 28, 1)
- Уменьшить количество фильтров (32, 64, 128 вместо 64, 128, 256)
- Упростить архитектуру (меньше слоев)
- Увеличить learning rate до 0.01
- Уменьшить dropout до 0.2-0.3

**CIFAR-100 (32x32, 3 канала, 100 классов):**
- Изменить количество выходных нейронов на 100
- Увеличить количество фильтров (128, 256, 512)
- Добавить больше слоев для сложности
- Использовать data augmentation
- Увеличить dropout до 0.4-0.5
- Добавить residual connections

**ImageNet (224x224, 3 канала, 1000 классов):**
- Изменить input_shape на (224, 224, 3)
- Использовать архитектуру ResNet, VGG или EfficientNet
- Добавить глобальный average pooling
- Использовать transfer learning с предобученными весами
- Применить data augmentation (rotation, zoom, flip)
- Использовать более сложные оптимизаторы (AdamW, SGD с momentum)
- Добавить learning rate scheduling
- Использовать mixed precision training


## Заключение

В данной лабораторной работе была реализована и проанализирована сверточная нейронная сеть для классификации изображений CIFAR-10. Основные достижения:

1. **Успешная реализация CNN** с архитектурой, включающей сверточные слои, пулинг, dropout и batch normalization
2. **Достижение приемлемой точности** на тестовом наборе данных
3. **Визуализация результатов** через confusion matrix и примеры предсказаний
4. **Анализ фильтров** первого сверточного слоя
5. **Улучшение модели** через модификацию архитектуры и гиперпараметров
6. **Сравнительный анализ** базовой и улучшенной моделей

Полученный опыт позволяет адаптировать архитектуру для работы с различными типами изображений и задачами классификации.
